# JOTME

## Features to Implement
```
1. Keyword Extraction using KeyBERT
    a. Draw a context graph of keywords
    b. Use context graph to do sentiment analysis
    c. Find negativity in the context
    d. Score the sentiment and store the entry and mood into database
    e. Return JSON response
        {
            "status": "success",
            "sentiment score": 0.7,
            "mood_score": [{"happy": 0.8}, {"sad": 0.2}, ...],
        }
2. Follow Ups based on previous entry
    a. Use GPT to generate follow-up questions when user writes next entry
```

In [1]:
# Import libaries
import spacy
import networkx as nx
from keybert import KeyBERT
from transformers import pipeline
import re

In [ ]:
class DiaryProcessor():
    def __init__(self):
        self.model = KeyBERT()
        self.nlp = spacy.load("en_core_web_sm")
        self.emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)
        print("Diary Processor initialized")

    def extract_keywords(self, text):
        top_n = max(10, text.count(' ') // 10 + 1)
        keywords = self.model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)

        # print(keywords)
        return [keyword[0] for keyword in keywords]
    
    def build_graph(self, text, keywords):
        G = nx.Graph()
        G.add_nodes_from(keywords)
        doc = self.nlp(text)
        sentences = [sent.text for sent in doc.sents]

        # Add edges between keywords that appear in the same sentence
        for sentence in sentences:
            sentence_keywords = [kw for kw in keywords if kw in sentence.lower()]
            for i in range(len(sentence_keywords)):
                for j in range(i + 1, len(sentence_keywords)):
                    G.add_edge(sentence_keywords[i], sentence_keywords[j])

        return G
    
    def analyze_emotions(self, text):
        results = self.emotion_classifier(text)[0]
        emotions = {result['label']: result['score'] for result in results}

        emotions = {emotion: score for emotion, score in emotions.items() if score >= 0.01}
        emotions = dict(sorted(emotions.items(), key=lambda item: item[1], reverse=True))
        total_score = sum(emotions.values())
        if total_score > 0:
            emotions = {emotion: score / total_score for emotion, score in emotions.items()}
        emotions = {emotion: round(score, 2) for emotion, score in emotions.items()}
        
        if not emotions:
            return {}  

        return emotions
    
    def process(self, text):
        keywords = self.extract_keywords(text)
        graph = self.build_graph(text, keywords)
        emotions = self.analyze_emotions(text)

        return keywords, graph, emotions

In [ ]:
test_texts = [
    "I forgot to confirm a meeting time—causing confusion and frustration. I apologized and learned to double-check calendar invites before sending. A small slip-up, big reminder to stay organized.",
    "Social media connects me to far-flung friends and ideas. But it often steals my morning peace. I'm trying 'social-fast Sundays'—no scrolling, just sunrise tea and a notebook.",
    "Woke up to gentle rain on the window. It set a calm tone for the day. Enjoyed a slow breakfast and felt grateful for the quiet morning.",
    "Felt a rush of excitement when a stranger complimented my artwork at the café. Small gestures brighten the day more than we expect.",
    "Stood up for myself in a meeting, even though my heart raced. Proud of speaking my truth and holding my ground.",
    "Took a moment today to observe the clouds drifting overhead. It reminded me how fleeting thoughts can be—just let them pass.",
    "Cooked a new dish today — turmeric-lime tofu. It turned out surprisingly well! I felt creative and nourished.",
    "Had a heartfelt call with an old friend. We laughed about childhood memories and plans—it felt like no time had passed.",
    "Ignored social media after morning scroll. Instead, I went for a walk and journaled. Felt grounded and peaceful.",
    "Missed a train by seconds, but the extra five minutes in the sun were peaceful. A delay turned into a quiet gift.",
    "Realized I haven't smiled enough lately. Made a point to smile at three strangers—each smile brought a spark of connection.",
    "Reflecting on love: it's less about fireworks and more about being there, day in and day out, without fanfare.",
]

diary_processor = DiaryProcessor()

for text in test_texts:
    keywords, graph, emotions = diary_processor.process(text)

    print("Keywords:", keywords)
    print("Emotions:", emotions)
    print("-" * 40)
    print()

Device set to use cuda:0


Diary Processor initialized
Keywords: ['calendar invites', 'confirm meeting', 'invites sending', 'invites', 'calendar', 'check calendar', 'meeting time', 'meeting', 'reminder', 'apologized']
Emotions: {'sadness': 0.41, 'anger': 0.38, 'neutral': 0.13, 'disgust': 0.04, 'surprise': 0.02, 'fear': 0.02}
----------------------------------------

Keywords: ['social fast', 'social media', 'sundays scrolling', 'social', 'fast sundays', 'trying social', 'scrolling just', 'friends ideas', 'sundays', 'scrolling']
Emotions: {'neutral': 0.89, 'sadness': 0.06, 'surprise': 0.02, 'disgust': 0.02, 'anger': 0.01}
----------------------------------------

Keywords: ['gentle rain', 'quiet morning', 'rain window', 'morning', 'woke gentle', 'breakfast felt', 'breakfast', 'rain', 'slow breakfast', 'woke']
Emotions: {'joy': 0.92, 'sadness': 0.04, 'neutral': 0.02, 'surprise': 0.01}
----------------------------------------

Keywords: ['artwork café', 'gestures brighten', 'complimented artwork', 'stranger complim